In [143]:
from konlpy.tag import Okt
import pandas as pd
import re
from collections import Counter

df= pd.read_csv('다음관객.csv')

df['sentiment']=0
df.loc[df['rating']==10, 'sentiment']=1
df.loc[df['rating']<=2, 'sentiment']=-1

In [149]:
#리뷰가 없는 데이터 제거
df= df[df['review'] != 'no review']
df

,rating,review,sentiment
0,10,현재는시골에서 열댓마리 강아지와 살고있다. 서울서는 지하에 내리 살았어서 그 냄...,1
1,9,유럽 친구가 내 의견 궁금하다고 꼭 봐 보래서 구입해서 봐 봤는데\n송강호가 이선균...,0
2,10,기분이 나빠진다.\n김선균에게 이입하고 싶지만 송강호인 현실이 슬프다.,1
3,10,시대의 명작. 평점이 낮은 이유는 저급한 댓글알바들과 봉준호를 싫어하는 ♪♬들이 여...,1
4,10,몇번이나 다시봐도 진짜 명작입니다,1
...,...,...,...
8655,1,평점이 좋은 이유를 모르겠네요. ㅎ\n영화보는 눈이 아둔해서 그런가\n무슨 메시지를...,-1
8656,1,이런게 영화면 성냥팔이 소녀의 재림은 신화다 ㅋ,-1
8657,2,별점 1점도 아깝다\n어떤 사람이 이 영화를 봤는데 처음에는 재미 있었지만 나중에는...,-1
8658,10,좌파폭도고발영화.전라도까는 내용이었어.,1


In [152]:
#10점인 review는 pos, 5점 이하인 review는 neg
from hanspell import spell_checker
okt= Okt()
stop_words= ['에','와','가','의','은','는','이다','이','가','로','을','으로','를','들']

pos_wrds= []
neg_wrds= []

for pos in df[df['sentiment']==1]['review'].to_list():
    npos= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',pos)
    spelled_sent= spell_checker.check(npos)
    han_pos= spelled_sent.checked
    wrd_list= okt.pos(han_pos,stem=True)
    for wrd,tag in wrd_list:
        if wrd not in stop_words and tag in ['Adverb','Adjective','Noun'] and len(wrd)>1:
            pos_wrds.append(wrd)
            
for neg in df[df['rating'] <= 5]['review'].to_list():
    new_neg= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',neg)
    spelled_sent= spell_checker.check(new_neg)
    han_neg= spelled_sent.checked
    wrd_list= okt.pos(han_neg,stem=True)
    for wrd,tag in wrd_list:
        if wrd not in stop_words and tag in ['Adverb','Adjective','Noun'] and len(wrd)>1:
            neg_wrds.append(wrd)

In [153]:
c=Counter(pos_wrds)
cm = dict(c.most_common(100))
print(cm)
print("\n")

c=Counter(neg_wrds)
cm = dict(c.most_common(100))
print(cm)

{'영화': 1994, '있다': 645, '생각': 435, '감독': 397, '없다': 386, '봉준호': 360, '최고': 342, '보고': 310, '좋다': 289, '너무': 276, '기생충': 259, '재밌다': 259, '같다': 240, '사람': 225, '현실': 212, '작품': 200, '정말': 198, '아니다': 186, '연기': 180, '많다': 175, '배우': 166, '사회': 164, '역시': 145, '한국': 143, '평점': 132, '축하': 129, '시간': 124, '재미있다': 122, '우리': 120, '장면': 120, '다시': 116, '그렇다': 113, '하나': 113, '여운': 112, '이렇다': 110, '불편하다': 104, '냄새': 103, '송강호': 97, '인간': 97, '슬프다': 96, '모두': 94, '재미': 93, '스토리': 89, '계속': 87, '천재': 84, '많이': 80, '몰입': 79, '어떻다': 78, '기대': 77, '대단하다': 75, '진짜': 74, '씁쓸하다': 73, '메시지': 71, '모든': 70, '이해': 69, '내용': 69, '느낌': 68, '세계': 68, '마음': 66, '의미': 66, '그냥': 65, '내내': 65, '정도': 64, '표현': 62, '훌륭하다': 61, '이야기': 61, '없이': 60, '가족': 60, '가지': 59, '계획': 58, '황금종려상': 58, '처음': 57, '아카데미': 57, '연출': 55, '마지막': 54, '인생': 53, '완벽하다': 53, '관객': 52, '다른': 51, '대한': 51, '가슴': 50, '시나리오': 47, '대박': 47, '명작': 46, '충격': 46, '깊다': 45, '반전': 45, '부자': 45, '이렇게': 45, '기생': 45, '여러': 44, '세상': 44, '소름': 43

In [165]:
from hanspell import spell_checker
pos_list= ['최고','재밌다','축하','재미있다','여운','몰입','대단하다','훌륭하다','완벽하다','대박','명작','자랑스럽다','소름','자랑','완벽','훌륭']
neg_list= ['별로','아깝다','재미없다','실망','쓰레기','억지','나쁘다','억지스럽다','최악','그다지','이상하다']

for rev in df[df['sentiment']==0]['review'].to_list():
    nrev= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',rev)
    spelled_sent= spell_checker.check(nrev)
    han_pos= spelled_sent.checked
    wrd_list= okt.morphs(han_pos,stem=True)
    pos_cnt= 0
    neg_cnt= 0
    for wrd in wrd_list:
        if wrd in pos_list:
            pos_cnt += 1
        elif wrd in neg_list:
            neg_cnt += 1
    if pos_cnt > neg_cnt:
        df.loc[df.review==rev, 'sentiment']= 1
    elif neg_cnt > pos_cnt:
        df.loc[df.review==rev, 'sentiment']=-1

In [166]:
df[df.sentiment==0].count()

rating       1948
review       1948
sentiment    1948
dtype: int64

In [168]:
from hanspell import spell_checker

cm_wrd=[]

for rev in df[df['sentiment']==0]['review'].to_list():
    nrev= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',rev)
    spelled_sent= spell_checker.check(nrev)
    han_pos= spelled_sent.checked
    wrd_list= okt.morphs(han_pos,stem=True)
    for wrd in wrd_list:
        if wrd not in stop_words and len(wrd)>1:
            cm_wrd.append(wrd)

c=Counter(cm_wrd)
cm = dict(c.most_common(100))
print(cm)

{'하다': 1443, '영화': 1150, '보다': 554, '있다': 460, '없다': 329, '되다': 322, '생각': 263, '않다': 257, '기생충': 233, '아니다': 224, '자다': 221, '같다': 219, '에서': 215, '사람': 195, '좋다': 195, '감독': 175, '보고': 166, '너무': 154, '현실': 144, '받다': 134, '사회': 133, '싶다': 129, '봉준호': 127, '그렇다': 126, '만들다': 125, '가족': 115, '불편하다': 114, '들다': 108, '느낌': 103, '하고': 102, '그냥': 102, '연기': 101, '냄새': 100, '가다': 99, '장면': 91, '많다': 91, '살다': 90, '나오다': 89, '에게': 85, '가난하다': 83, '모르다': 81, '나다': 80, '우리': 78, '송강호': 76, '재미': 76, '이해': 74, '작품': 71, '주다': 71, '스토리': 69, '인간': 69, '내용': 68, '이렇다': 66, '배우': 66, '먹다': 65, '보여주다': 62, '씁쓸하다': 61, '마지막': 61, '부자': 61, '그리고': 61, '까지': 60, '이고': 60, '시간': 59, '정말': 59, '느끼다': 59, '메시지': 57, '대한': 56, '기생': 56, '다른': 55, '하나': 55, '기대': 53, '모습': 53, '한국': 52, '역시': 52, '모두': 52, '부분': 52, '처럼': 50, '정도': 50, '많이': 50, '표현': 49, '오다': 49, '기분': 49, '뭔가': 49, '전개': 49, '평점': 48, '설정': 48, '인가': 48, '되어다': 48, '맞다': 48, '내내': 48, '어떻다': 48, '살인': 47, '자본주의': 47, '계층': 46, '관람': 46

In [172]:
#별점 기준 데이터 선별
from hanspell import spell_checker

for rev in df[df['sentiment']==0]['review'].to_list():
    nrev= re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',rev)
    spelled_sent= spell_checker.check(nrev)
    han_pos= spelled_sent.checked
    wrd_list= okt.morphs(han_pos,stem=True)
    if '하다' in wrd_list or '영화' in wrd_list or '보다' in wrd_list or '있다' in wrd_list or '없다' in wrd_list or '되다' in wrd_list or '생각' in wrd_list and df[df['review']==rev]['rating'].mean() >= 8:
        df.loc[df.review==rev, 'sentiment']= 1
    elif '하다' in wrd_list or '영화' in wrd_list or '보다' in wrd_list or '있다' in wrd_list or '없다' in wrd_list or '되다' in wrd_list or '생각' in wrd_list and df[df['review']==rev]['rating'].mean() <= 5:
        df.loc[df.review==rev, 'sentiment']= -1
    elif '않다' in wrd_list or '아니다' in wrd_list or '같다' in wrd_list or '기생충' in wrd_list or '사람' in wrd_list or '감독' in wrd_list or '현실' in wrd_list and df[df['review']==rev]['rating'].mean() >= 8:
        df.loc[df.review==rev, 'sentiment']=1
    elif '않다' in wrd_list or '아니다' in wrd_list or '같다' in wrd_list or '기생충' in wrd_list or '사람' in wrd_list or '감독' in wrd_list or '현실' in wrd_list and df[df['review']==rev]['rating'].mean() <= 5:
        df.loc[df.review==rev, 'sentiment']=-1

In [173]:
df[df.sentiment==0].count()

rating       457
review       457
sentiment    457
dtype: int64

In [175]:
df.to_csv("다음관객_수정.csv")